## Imports

In [21]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula

## Read-in

In [45]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

## Clean Data

In [46]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [47]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [48]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [49]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [50]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [53]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,geocoded,lat,lon
1,Estates at Acqualina\r17975 Collins Ave 1202\r...,3/22/2023,4,"$8,950,000.00",N/A,"$2,041",Yansy Checa,Acqualina Realty,Adriana Vargas Hernandez,Acqualina Realty,Estates at Acqualina 17975 Collins Ave 1202 Su...,Estates at Acqualina,17975 Collins Ave 1202 Sunny Isles Beach,"(25.9423323, -80.12016559999999)",25.942332,-80.120166
2,One Paraiso\r3131 NE 7th Ave PH5203\rMiami,3/20/2023,31,"$3,950,000.00",2887,"$1,368.20",Liz Hogan,"Compass Florida, LLC",Peter Ancona,Ancona Real Estate Inc,One Paraiso 3131 NE 7th Ave PH5203 Miami,One Paraiso,3131 NE 7th Ave PH5203 Miami,"(25.8071448, -80.1856208)",25.807145,-80.185621
3,Jade Signature\r16901 Collins Ave 2903\rSunny ...,3/22/2023,371,"$3,750,000.00",2511,"$1,493.43",Bruno Ricci,Fortune Christie's Int'l Real Estate,Roberto Viana Tucunduva,Elite International Realty Inc,Jade Signature 16901 Collins Ave 2903 Sunny Is...,Jade Signature,16901 Collins Ave 2903 Sunny Isles Beach,"(25.9327557, -80.1210054)",25.932756,-80.121005
4,Roney Palace Condo\r2301 Collins Ave 609\rMiam...,3/22/2023,643,"$2,690,000.00",1480,"$1,817.57",Tessa Trlaja,COSMORE Florida Corp.,Sophia Albrecht,Thine Agency,Roney Palace Condo 2301 Collins Ave 609 Miami ...,Roney Palace Condo,2301 Collins Ave 609 Miami Beach,"(25.79871, -80.1269026)",25.798710,-80.126903
5,Gables on the Green\r626 Coral Way LPH03\rCora...,3/20/2023,356,"$2,600,000.00",5402,$632.45,Mauricio Barba PA,"Compass Florida, LLC",Marcello Agostini,"AG Real Estate Advisors, LLC",Gables on the Green 626 Coral Way LPH03 Coral ...,Gables on the Green,626 Coral Way LPH03 Coral Gables,"(25.7489953, -80.26794749999999)",25.748995,-80.267947
6,Grovenor House\r2627 S Bayshore Dr 1606\rMiami,3/20/2023,113,"$2,300,000.00",1756,"$1,309.80",Jorge Rico,"Compass Florida, LLC",Geraldine Di Lorenzo,Optimar International Realty,Grovenor House 2627 S Bayshore Dr 1606 Miami,Grovenor House,2627 S Bayshore Dr 1606 Miami,"(25.7322307, -80.2348947)",25.732231,-80.234895
7,Green Diamond Condo\r4775 Collins Ave 1202\rMi...,3/21/2023,139,"$2,150,000.00",1530,"$1,405.23",Romina Tripichio,Miami Beach Investment Realty,Laura Curson,Coldwell Banker Realty,Green Diamond Condo 4775 Collins Ave 1202 Miam...,Green Diamond Condo,4775 Collins Ave 1202 Miami Beach,"(25.8225276, -80.1215168)",25.822528,-80.121517
8,Ocean Three\r18911 Collins Ave 905\rSunny Isle...,3/24/2023,367,"$2,050,000.00",2864,$715.78,Ludmila Bogatov,Bogatov Realty,Elizabeth Rodriguez,London Foster Realty,Ocean Three 18911 Collins Ave 905 Sunny Isles ...,Ocean Three,18911 Collins Ave 905 Sunny Isles Beach,"(25.9508781, -80.1199875)",25.950878,-80.119987
9,Trump Tower Two\r15901 Collins Ave 3504\rSunny...,3/21/2023,186,"$1,850,000.00",2327,$795.02,Lana Bell,Douglas Elliman,Marco Cunha,Prestige Luxury Real Estate,Trump Tower Two 15901 Collins Ave 3504 Sunny I...,Trump Tower Two,15901 Collins Ave 3504 Sunny Isles Beach,"(25.923688, -80.12177)",25.923688,-80.121770
10,0Paraiso\r650 NE 32nd St PH5002\rMiami,3/21/2023,30,"$1,800,000.00",1808,$995.58,Silvia Olivera PA,"FIP Realty Services, LLC",Paul Lobosco,Amerigo International,0Paraiso 650 NE 32nd St PH5002 Miami,Paraiso,650 NE 32nd Street,"(25.8069485, -80.1862675)",25.806949,-80.186267


In [52]:
df.at[1,'price_per_sqft']=('$2,041')

df.at[10,'building_name']=('Paraiso')
df.at[10,'address_only']=('650 NE 32nd Street')

## Format Data

In [54]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

In [55]:
df['int_Sale_Price'] = df['Sale Price'].str.replace('$','',regex=True)
df['int_Sale_Price'] = df['int_Sale_Price'].str.replace(',','',regex=True)
df['int_Sale_Price'] = pd.to_numeric(df['int_Sale_Price'])

df['price_per_sqft'] = df['price_per_sqft'].str.replace('$','',regex=True)
df['price_per_sqft'] = df['price_per_sqft'].str.replace(',','',regex=True)
df['price_per_sqft'] = pd.to_numeric(df['price_per_sqft'])

df['days_on_market'] = pd.to_numeric(df['days_on_market'])

## Color-code top sale

In [56]:
### Insert RANK values ###
df['RANK'] = range(1, len(df) + 1)
# use numpy to assign values to the 'COLOR' column
df['COLOR'] = np.where(df['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [58]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

In [59]:
def popup_html(row):
    building_name = row['building_name']
    price = row['Sale Price']
    days_on_market = row['days_on_market']
    listing_agent = row['Agent']
    buyers_agent = row['Buyer Agent']
    psf = row['Sq. Ft.']
    address = row['address_only']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>{}'''.format(price) + '''<br>
    <strong>Days on Market: </strong>{}'''.format(days_on_market) + '''<br>
    <strong>Listing Agent: </strong>{}'''.format(listing_agent) + '''<br>
    <strong>Buyer's Agent: </strong>{}'''.format(buyers_agent) + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(psf) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [60]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('March 19th - March 26th')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [37]:
m.save('index.html')

## Data snagger

In [38]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [39]:
### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['int_Sale_Price'].idxmax()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmax()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmax()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['int_Sale_Price'].idxmin()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmin()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmin()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")

Most Expensive
Estates at Acqualina , 17975 Collins Ave 1202 Sunny Isles Beach | Price $8,950,000 | $2,041 psf | Listing agent: Yansy Checa with Acqualina Realty | Buyer's agent: Adriana Vargas Hernandez with Acqualina Realty | Days on market: 4
Least Expensive
Paraiso, 650 NE 32nd Street | Price $1,800,000 | $996 psf | Listing agent: Silvia Olivera PA with FIP Realty Services, LLC | Buyer's agent: Paul Lobosco with Amerigo International | Days on market: 30
Highest Price Per Square Foot
Estates at Acqualina , 17975 Collins Ave 1202 Sunny Isles Beach | Price $8,950,000 | $2,041 psf | Listing agent: Yansy Checa with Acqualina Realty | Buyer's agent: Adriana Vargas Hernandez with Acqualina Realty | Days on market: 4
Lowest Price Per Square Foot
Gables on the Green , 626 Coral Way LPH03 Coral Gables | Price $2,600,000 | $632 psf | Listing agent: Mauricio Barba PA with Compass Florida, LLC | Buyer's agent: Marcello Agostini with AG Real Estate Advisors, LLC | Days on market: 356
Most Days 

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [40]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [41]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_03272023
